In [ ]:
import pandas as pd
import os
import matplotlib.pyplot as plt
from PIL import Image

from medical_image_segmentation.analyze_data.utils import get_file_paths

In [ ]:
root_dir = "/scratch/gpfs/RUSTOW/med_datasets/nih_chest_x_rays"

In [ ]:
csv_path = os.path.join(root_dir, "Data_Entry_2017.csv")
df = pd.read_csv(csv_path)

In [ ]:
images_list = get_file_paths(root_dir, lambda x: x.endswith(".png"))

In [ ]:
image_to_label_map = dict(zip(df["Image Index"], df["Finding Labels"]))

In [ ]:
label_count = {}
for image, labels in image_to_label_map.items():
    if labels not in label_count:
        label_count[labels] = 1
    else:
        label_count[labels] += 1

In [ ]:
kept_labels = []
for label, count in label_count.items():
    if count >= 1000 and "|" not in label:
        kept_labels.append(label)

print(kept_labels)

In [ ]:
image_index_to_image_path_map = {}
for image_path in images_list:
    index = image_path.split("/")[-1]
    image_index_to_image_path_map[index] = image_path

In [ ]:
df["Image Path"] = df["Image Index"].map(image_index_to_image_path_map)

In [ ]:
df = df[df["Finding Labels"].isin(kept_labels)]

In [ ]:
df = df.groupby('Finding Labels').sample(n=1000, random_state=42)
print(df['Finding Labels'].value_counts())

In [ ]:
df = df[["Finding Labels", "Image Path", "Image Index"]].reset_index(drop=True)
df = df.rename(columns={"Finding Labels": "label", "Image Path": "image_path", "Image Index": "image_index"})

In [ ]:
df

In [ ]:
output_path = "/scratch/gpfs/eh0560/repos/medical-image-segmentation/data/nih_chest_x_ray_subset_info/original_image_path_to_label.csv"
df.to_csv(output_path, index=False)